In [1]:
import add_packages
import psycopg2
import os

In [2]:
SQL_DB = os.getenv("SQL_DB")
SQL_HOST = os.getenv("SQL_HOST")
SQL_USER = os.getenv("SQL_USER")
SQL_PASSWORD = os.getenv("SQL_PASSWORD")
SQL_PORT = os.getenv("SQL_PORT")

In [4]:
conn = psycopg2.connect(
	database=SQL_DB, 
	host=SQL_HOST, 
	user=SQL_USER,
	password=SQL_PASSWORD,
	port=SQL_PORT,
)

cursor = conn.cursor()

conn.close()
cursor.close()
conn.close()

In [8]:
import logging
from typing import Literal
import psycopg2
from psycopg2.extras import RealDictCursor
from psycopg2.pool import SimpleConnectionPool

class SQLDatabase:
	def __init__(
		self,
		dbname: str = os.getenv("SQL_DB"),
		user: str = os.getenv("SQL_USER"),
		password: str = os.getenv("SQL_PASSWORD"),
		host: str = os.getenv("SQL_HOST"),
		port: str = os.getenv("SQL_PORT"),
		min_conn: int = 1,
		max_conn: int = 1,
	):
		self.dbname = dbname
		self.user = user
		self.password = password
		self.host = host
		self.port = port
		self.logger = logging.getLogger(__name__)

		self.pool = SimpleConnectionPool(
			minconn=min_conn,
			maxconn=max_conn,
			dsn="dbname={dbname} user={user} password={password} host={host} port={port}".format(
				dbname=dbname, user=user, password=password, host=host, port=port
			)
		)

	def execute_query(
			self,
			query: str,
			params: tuple = None,
			fetch_option: Literal["one", "many", "all"] = "all",
    ):
			conn: psycopg2.extensions.connection = None
			cur: RealDictCursor = None
			try:
					conn = self.pool.getconn()
					cur = conn.cursor(cursor_factory=RealDictCursor)
					if params:
							cur.execute(query, params)
					else:
							cur.execute(query)
					if fetch_option == "one":
							return cur.fetchone()
					elif fetch_option == "many":
							return cur.fetchmany()
					elif fetch_option == "all":
							return cur.fetchall()
			except (Exception, psycopg2.DatabaseError) as error:
					self.logger.error(f"Error: {error}")
					return None
			finally:
					if cur is not None:
							cur.close()
					if conn is not None:
							self.pool.putconn(conn)

	def execute_commit(self, query, params=None):
		conn: psycopg2.extensions.connection = None
		cur: RealDictCursor = None
		try:
			conn = self.pool.getconn()
			cur = conn.cursor()
			if params:
				cur.execute(query, params)
			else:
				cur.execute(query)
			conn.commit()
			return cur.rowcount
		except (Exception, psycopg2.DatabaseError) as error:
			self.logger.error(f"Error: {error}")
			if conn is not None:
				conn.rollback()
			return None
		finally:
			if cur is not None:
				cur.close()
			if conn is not None:
				self.pool.putconn(conn)

In [65]:
import json
import datetime
import pytz
from dateutil import tz

class SQLTable:
    def __init__(self, name: str, schema: list, db: SQLDatabase):
        self.name = name
        self.schema = schema
        self.db = db

        # Extract column names from schema and store them in a separate list
        self.col_names = [col.split()[0] for col in schema if col.split()[0] not in ["PRIMARY", "FOREIGN"]]

        # Store the timezone of the local system
        self.tz = datetime.timezone.utc if datetime.timezone.utc.tzname(None) == "UTC" else datetime.timezone(datetime.timedelta(seconds=time.timezonezone(None)))

    def create(self):
        query = f"CREATE TABLE IF NOT EXISTS {self.name} ("
        for i, col in enumerate(self.schema):
            query += f"{col} "
            if i < len(self.schema) - 1:
                query += ", "
        query += ")"
        self.db.execute_commit(query)

    def insert(self, data: dict):
        # Exclude the 'id SERIAL', 'created_at', and 'updated_at' columns from the list of columns and placeholders
        cols = ", ".join(col for col in self.col_names if col not in ["id", "created_at", "updated_at"])
        placeholders = ", ".join("%s" for col in self.col_names if col not in ["id", "created_at", "updated_at"])

        # Convert the timestamps to JSON objects with a timezone offset
        now = datetime.datetime.now(self.tz)
        data["created_at"] = self._to_json_date(now)
        data["updated_at"] = self._to_json_date(now)

        query = f"INSERT INTO {self.name} ({cols}) VALUES ({placeholders})"
        self.db.execute_commit(query, tuple(data[col] for col in self.col_names if col not in ["id", "created_at", "updated_at"]))

    def delete_by_col(self, col: str, value: str):
        """Delete rows from the table that match a given column value."""
        condition = f"{col} = %s"
        params = (value,)
        self.delete(condition, params)

    def update_by_col(self, col: str, value: str, data: dict):
        """Update rows in the table that match a given column value."""
        condition = f"{col} = %s"
        params = (value,) + tuple(data[col] for col in data)
        self.update(data, condition, params)

    def delete(self, condition: str, params: tuple = None):
        query = f"DELETE FROM {self.name} WHERE {condition}"
        self.db.execute_commit(query, params)

    def update(self, data: dict, condition: str, params: tuple = None):
        cols = ", ".join(f"{col}=%s" for col in data)
        query = f"UPDATE {self.name} SET {cols} WHERE {condition}"
        self.db.execute_commit(query, params + tuple(data[col] for col in data))

    def query(self, query: str, params: tuple = None, fetch_option: Literal["one", "many", "all"] = "all"):
        cur = self.db.execute_query(query, params)
        if fetch_option == "one":
            return cur.fetchone()
        elif fetch_option == "many":
            return cur.fetchmany()
        elif fetch_option == "all":
            return cur.fetchall()

    def _to_json_date(self, dt: datetime.datetime):
        """Convert a datetime.datetime object to a JSON object with a timezone offset."""
        return json.dumps({"$date": dt.strftime("%Y-%m-%dT%H:%M:%S.%f")[:-3] + self.tz.tzname(None)})

    def _from_json_date(self, json_str: str):
        """Convert a JSON object with a timezone offset to a datetime.datetime object."""
        obj = json.loads(json_str)
        dt_str = obj["$date"][:-6] + obj["$date"][-5:]
        dt = datetime.datetime.strptime(dt_str, "%Y-%m-%dT%H:%M:%S.%f")
        tz_str = obj["$date"][-6:]
        tz = pytz.timezone(tz_str) if tz_str != "UTC" else datetime.timezone.utc
        return dt.replace(tzinfo=tz)


In [9]:
my_sql = SQLDatabase()

In [10]:
my_sql.execute_commit(
	"""CREATE TABLE datacamp_courses(
	course_id SERIAL PRIMARY KEY,
	course_name VARCHAR (50) UNIQUE NOT NULL,
	course_instructor VARCHAR (100) NOT NULL,
	topic VARCHAR (20) NOT NULL);
	"""
)

-1

In [66]:
users_schema = [
	"id SERIAL",
	"username VARCHAR(50) UNIQUE NOT NULL",
	"email VARCHAR(255) UNIQUE NOT NULL",
	"password_hash VARCHAR(128) NOT NULL",
	"created_at TIMESTAMP DEFAULT NOW()",
	"updated_at TIMESTAMP DEFAULT NOW()",
	"PRIMARY KEY (id)"
]

users_table = SQLTable(
	name="users", 
	schema=users_schema,
	db=my_sql
)

users_table.create()

In [64]:
new_user = {
	"username": "johndoe",
	"email": "johndoe@example.com",
	"password_hash": "12345678"  # This should be a hashed version of the user's password
}

users_table.insert(new_user)

In [69]:
# Delete a user with the email "johndoeeee@example.com"
users_table.delete_by_col("email", "johndoee@example.com")

In [70]:
# Update a user with the username "johndoe" and email "johndoe@example.com"
data = {
	"username": "johndoe_new",
	"email": "johndoe_new@example.com",
	"password_hash": "12345678"  # This should be a hashed version of the user's password
}
users_table.update_by_col("username", "johndoe", data)

Error: not all arguments converted during string formatting


# Backup

In [ ]:
import json
import datetime
import pytz
from dateutil import tz

class SQLTable:
    def __init__(self, name: str, schema: list, db: SQLDatabase):
        self.name = name
        self.schema = schema
        self.db = db

        # Extract column names from schema and store them in a separate list
        self.col_names = [col.split()[0] for col in schema if col.split()[0] not in ["PRIMARY", "FOREIGN"]]

        # Store the timezone of the local system
        self.tz = datetime.timezone.utc if datetime.timezone.utc.tzname(None) == "UTC" else datetime.timezone(datetime.timedelta(seconds=time.timezonezone(None)))

    def create(self):
        query = f"CREATE TABLE IF NOT EXISTS {self.name} ("
        for i, col in enumerate(self.schema):
            query += f"{col} "
            if i < len(self.schema) - 1:
                query += ", "
        query += ")"
        self.db.execute_commit(query)

    def insert(self, data: dict):
        # Exclude the 'id SERIAL', 'created_at', and 'updated_at' columns from the list of columns and placeholders
        cols = ", ".join(col for col in self.col_names if col not in ["id", "created_at", "updated_at"])
        placeholders = ", ".join("%s" for col in self.col_names if col not in ["id", "created_at", "updated_at"])

        # Convert the timestamps to JSON objects with a timezone offset
        now = datetime.datetime.now(self.tz)
        data["created_at"] = self._to_json_date(now)
        data["updated_at"] = self._to_json_date(now)

        query = f"INSERT INTO {self.name} ({cols}) VALUES ({placeholders})"
        self.db.execute_commit(query, tuple(data[col] for col in self.col_names if col not in ["id", "created_at", "updated_at"]))

    def delete_by_col(self, col: str, value: str):
        """Delete rows from the table that match a given column value."""
        condition = f"{col} = %s"
        params = (value,)
        self.delete(condition, params)

    def update_by_col(self, col: str, value: str, data: dict):
        """Update rows in the table that match a given column value."""
        condition = f"{col} = %s"
        params = (value,) + tuple(data[col] for col in data)
        self.update(data, condition, params)

    def delete(self, condition: str, params: tuple = None):
        query = f"DELETE FROM {self.name} WHERE {condition}"
        self.db.execute_commit(query, params)

    def update(self, data: dict, condition: str, params: tuple = None):
        cols = ", ".join(f"{col}=%s" for col in data)
        query = f"UPDATE {self.name} SET {cols} WHERE {condition}"
        self.db.execute_commit(query, params + tuple(data[col] for col in data))

    def query(self, query: str, params: tuple = None, fetch_option: Literal["one", "many", "all"] = "all"):
        cur = self.db.execute_query(query, params)
        if fetch_option == "one":
            return cur.fetchone()
        elif fetch_option == "many":
            return cur.fetchmany()
        elif fetch_option == "all":
            return cur.fetchall()

    def _to_json_date(self, dt: datetime.datetime):
        """Convert a datetime.datetime object to a JSON object with a timezone offset."""
        return json.dumps({"$date": dt.strftime("%Y-%m-%dT%H:%M:%S.%f")[:-3] + self.tz.tzname(None)})

    def _from_json_date(self, json_str: str):
        """Convert a JSON object with a timezone offset to a datetime.datetime object."""
        obj = json.loads(json_str)
        dt_str = obj["$date"][:-6] + obj["$date"][-5:]
        dt = datetime.datetime.strptime(dt_str, "%Y-%m-%dT%H:%M:%S.%f")
        tz_str = obj["$date"][-6:]
        tz = pytz.timezone(tz_str) if tz_str != "UTC" else datetime.timezone.utc
        return dt.replace(tzinfo=tz)
